# Grupo 03 - Desafío 01
## Estrategia de construcción
>**OBJETIVO:** Crear un df de post-producción que luego se transformará en producción para realizar análisis del dataset.
> * Se trabajará depurando cada columna del dataset con la finalidad de reconstruir datos perdidos y/o crear nuevas columnas que serán utilizadas a posteriori en la fase de análisis.

### Instalación de librerías

In [ ]:
#!pip install unidecode

### Importación de librerías

In [1]:
import sys
sys.path.append("../src")
import g3utils as g3

import pandas as pd
import numpy as np
import re as re
import unidecode

### Carga e inicialización

In [2]:
# Carga de dataset inicial

df = pd.DataFrame(pd.read_csv("../data/properatti_minusculas.csv", encoding='UTF-8'))
#!head nombreArchivo.cvs --> comando de linux para ver las 5 primeras lineas. Se puede utilizar para ver como estan separados los datos

In [3]:
# Definición de cantidad de filas a cargar en caso de realizar una prueba acotada

#ca_filas = 40
ca_filas = 121220

In [4]:
# Reemplado de NaN's para comenzar a estandarizar los datos ingresados en data frame de inicio

df = g3.reemplaza_nan('sin datos',df)

In [5]:
# Verificación del formato del data frame incicial

df.shape

(121220, 27)

### Armado inicial de data frames operativos
Este bloque se reserva para realizar la precarga inicial de distintos data frames que se utilizarán a lo largo del notebook.
* pospo -- df de posproducción
* df_recup -- df para persistir los elementos recuperados


In [6]:
# Dadtaframe de posproducción. Tras depurar todas las regex, este DF será la base de análisis de negocio
# Se ejecuta una única vez y se le van incorporando las columnas depuradas.
# Tener en cuenta que el DF pospo definitivo tiene que tener la misma cantidad de filas que el DF base len(df)

pospo = g3.generar_df_posproduccion(ca_filas)
pospo.shape
#pospo

(121220, 1)

In [7]:
# Definición de df que permite registrar los valores recuperados de cada columna

df_recup = pd.DataFrame()
df_recup.shape

(0, 0)

### Procesamiento de carga de columnas depuradas en data frame post-produccion 'pospo'
> *** Modo de uso:** 
>* En este markdown se vuelcan las funciones que generan y agregan columnas depuradas dentro de pospo
>* La codificación de desarrollo se realiza en notebook de **Desarrollo** y una vez terminado se empaqueta dentro de una función consolidadora para automatizar tareas de carga.



In [8]:
# Depuración 'ambientes' en 'description', persistencia de columna en pospro y recup

def recuperar_ambientes(pospo, recup):
    pattern = r'(\d*\w*)\s*amb'  
    
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas))
    dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
    
    df_temp = g3.limpiar_columna_x_clave(dic, df1)
    recuperados = df_temp[pd.notnull(df_temp[0])].count() # selecciona unicamente los recuperados del regex
    recup = g3.registrar_recupero(int(recuperados),'r_ambientes', recup)
    pospo = g3.agregar_columna('r_ambiente', pospo, df_temp)
    return pospo, recup

In [9]:
pospo, df_recup = recuperar_ambientes(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (121220, 2)
df_recup.size ==> (1, 1)


In [10]:
# Depuración 'precio usd' en 'description', persistencia de columna en pospro y recup

def recuperar_usd(pospo, recup):
    pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?u[$sd]\w?|u[$sd]\w?\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio_usd
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado(df1, 'r_usd') # Genera DF acotado con los valores recuperados unicamente

    recup = g3.registrar_recupero(len(df2),'r_usd', recup)

    pospo = pd.merge(pospo, df2, on='indice', how='left')
    return pospo, recup

In [11]:
pospo, df_recup = recuperar_usd(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (121220, 3)
df_recup.size ==> (1, 2)


In [12]:
# Depuración 'metros2' en 'description' y 'title', persistencia de columna en pospro y recup

def recuperar_metros(pospo, recup):
    pattern = r"(\d*)\,?\d*\s?(?:m²|mts\s?2|metros\s?2|mts²|m2|metros\s?cuadrado|mts\s?cuadrado)" # superficie
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'r_metros2') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_simple(df3, 'r_metros2')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_metros2', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [13]:
pospo, df_recup = recuperar_metros(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (121220, 4)
df_recup.size ==> (1, 3)


In [14]:
# Depuración 'frente' en 'description' y 'title', persistencia de columna en pospro y recup

def recuperar_frente(pospo, recup):
    pattern = r"(frente)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_frente')
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_booleano(df3, 'r_frente')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_frente', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [15]:
pospo, df_recup = recuperar_frente(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (121220, 5)
df_recup.size ==> (1, 4)


In [16]:
# Depuración 'contrafrente' en 'description' y 'title', persistencia de columna en pospro y recup

def recuperar_contrafrente(pospo, recup):
    pattern = r"(contrafrente)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas))
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_contrafrente')
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_booleano(df3, 'r_contrafrente')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_contrafrente', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [17]:
pospo, df_recup = recuperar_contrafrente(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (121220, 6)
df_recup.size ==> (1, 5)


In [18]:
# Depuración 'expensas' en 'description' y 'title', persistencia de columna en pospro y recup

def recuperar_expensas(pospo, recup):
    pattern = r"(expensas)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_expensas')
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_booleano(df3, 'r_expensas')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
        
    recup = g3.registrar_recupero(len(df5),'r_expensas', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup


In [19]:
pospo, df_recup = recuperar_expensas(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (121220, 7)
df_recup.size ==> (1, 6)


In [20]:
def recuperar_garage(pospo, recup):
    pattern = r"(garage)|(cochera)" # garage
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_garage') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_booleano(df3, 'r_garage')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_garage', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [21]:
pospo, df_recup = recuperar_garage(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (121220, 8)
df_recup.size ==> (1, 7)


In [22]:
def recuperar_amenities(pospo, recup):
    pattern = r"(pileta)|(piscina)|(SUM)|(laundry)|(lavadero)|(terraza)|(solarium)|(baulera)|(sauna)|(gimnasio)|(salon de usos multiples)|(cochera)|(garage)|(gim)|(gym)" # garage
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_amenities') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_booleano(df3, 'r_amenities')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_amenities', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [23]:
pospo, df_recup = recuperar_amenities(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (121220, 9)
df_recup.size ==> (1, 8)


In [24]:
df_recup

,r_ambientes,r_usd,r_metros2,r_frente,r_contrafrente,r_expensas,r_garage,r_amenities
0,47939,12820,40073,43335,9664,16732,56971,96318


In [25]:
pospo

,indice,r_ambiente,r_usd,r_metros2,r_frente,r_contrafrente,r_expensas,r_garage,r_amenities
0,0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0
1,1,NaN,20.000,NaN,1.0,NaN,NaN,1.0,1.0
2,2,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0
3,3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
5,5,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
6,6,NaN,NaN,106,1.0,NaN,1.0,NaN,1.0
7,7,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,8,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


### Incorporación de características originales
* Dichas características se extraen del data frame 'df' y se persisten en 'pospo'.
* Las características seleccionadas fueron analizadas previamente y solo se persistirán las necesarias.

In [27]:
g3.crear_csv('../data/recuperados_properati.csv',df_recup)

[LOG] Se ha creado el archivo:  ../data/recuperados_properati.csv


In [26]:
g3.crear_csv('../data/pospo_properati.csv',pospo)

[LOG] Se ha creado el archivo:  ../data/pospo_properati.csv


In [28]:
pospo_bkp = pospo